In [9]:
# import tensorflow as tf
import numpy as np
import h2o
import os
import pandas as pd
from h2o.grid.grid_search import H2OGridSearch


from concurrent.futures import ThreadPoolExecutor, as_completed
# from .feature_engineering_filter import Find_correct_port

In [10]:
train_data = pd.read_csv("../Datasets/ais_train.csv", delimiter="|")
test_data = pd.read_csv("../Datasets/ais_test.csv", delimiter=",")

In [11]:
vessel_data = pd.read_csv("../Datasets/vessels.csv", delimiter="|")
port_data = pd.read_csv("../Datasets/ports.csv", delimiter="|")

In [12]:
train_data["time"] = pd.to_datetime(train_data["time"])
test_data["time"] = pd.to_datetime(test_data["time"])

In [13]:
#train_data=train_data.merge(vessel_data[['vesselId', 'shippingLineId']], on='vesselId', how='left')

In [14]:
port_data_renamed=pd.DataFrame()
port_data_renamed[["portId","port_latitude","port_longitude"]]=port_data[["portId","latitude","longitude"]]
train_data=train_data.merge(port_data_renamed, on="portId", how="left")

In [15]:
print(train_data.columns)

Index(['time', 'cog', 'sog', 'rot', 'heading', 'navstat', 'etaRaw', 'latitude',
       'longitude', 'vesselId', 'portId', 'port_latitude', 'port_longitude'],
      dtype='object')


In [16]:
train_data_preprocessed = train_data
train_data_preprocessed.loc[train_data_preprocessed["cog"] >= 360, "cog"] = np.nan
train_data_preprocessed.loc[train_data_preprocessed["sog"] >= 1023, "sog"] = np.nan
train_data_preprocessed.loc[train_data_preprocessed["rot"] == -128, "rot"] = np.nan
train_data_preprocessed.loc[train_data_preprocessed["heading"] == 511, "heading"] = (
    np.nan
)


pattern = r"^\d{2}-\d{2} \d{2}:\d{2}$"
train_data_preprocessed["etaRaw"] = train_data_preprocessed["etaRaw"].where(
    train_data_preprocessed["etaRaw"].str.match(pattern, na=False), np.nan
)


train_data_preprocessed = train_data_preprocessed.sort_values("time")

print(train_data_preprocessed.head())


train_data_preprocessed = (
    train_data_preprocessed.groupby("vesselId")
    .apply(lambda group: group.ffill().bfill())
    .reset_index(drop=True)
)


print(train_data_preprocessed.head())

train_data_preprocessed["heading"] = train_data_preprocessed["heading"].fillna(0)

train_data_preprocessed = train_data_preprocessed.dropna().reset_index(drop=True)


# Replace '00-' in etaRaw with the corresponding month and day from the 'time' column
train_data_preprocessed["etaRaw"] = train_data_preprocessed["etaRaw"].mask(
    train_data_preprocessed["etaRaw"].str.contains("00-", na=False),
    "01" + train_data_preprocessed["etaRaw"].str[2:],
)

train_data_preprocessed["etaRaw"] = train_data_preprocessed["etaRaw"].mask(
    train_data_preprocessed["etaRaw"].str.contains("-00", na=False),
    train_data_preprocessed["etaRaw"].str[:2]
    + "-01"
    + train_data_preprocessed["etaRaw"].str[5:],
)

train_data_preprocessed["etaRaw"] = train_data_preprocessed["etaRaw"].mask(
    train_data_preprocessed["etaRaw"].str.contains(":60", na=False),
    train_data_preprocessed["etaRaw"].str[:9] + "59",
)

train_data_preprocessed["etaRaw"] = train_data_preprocessed["etaRaw"].mask(
    train_data_preprocessed["etaRaw"].str.contains("60:", na=False),
    train_data_preprocessed["etaRaw"].str[:6] + "01:00",
)

train_data_preprocessed["etaRaw"] = train_data_preprocessed["etaRaw"].mask(
    train_data_preprocessed["etaRaw"].str.contains("24:", na=False),
    train_data_preprocessed["etaRaw"].str[:6] + "23:59",
)


train_data_preprocessed["etaRaw"] = pd.to_datetime(
    train_data_preprocessed["time"].dt.year.astype(str)
    + "-"
    + train_data_preprocessed["etaRaw"]
    + ":00",
    format="%Y-%m-%d %H:%M:%S",
)


train_data_preprocessed["seconds_to_eta"] = (
    train_data_preprocessed["etaRaw"] - train_data_preprocessed["time"]
).dt.total_seconds()

train_data_preprocessed = train_data_preprocessed.drop(columns=["etaRaw"])

                 time    cog   sog  rot  heading  navstat       etaRaw  \
0 2024-01-01 00:00:25  284.0   0.7  0.0     88.0        0  01-09 23:00   
1 2024-01-01 00:00:36  109.6   0.0 -6.0    347.0        1  12-29 20:00   
2 2024-01-01 00:01:45  111.0  11.0  0.0    112.0        0  01-02 09:00   
3 2024-01-01 00:03:11   96.4   0.0  0.0    142.0        1  12-31 20:00   
4 2024-01-01 00:03:51  214.0  19.7  0.0    215.0        0  01-25 12:00   

   latitude  longitude                  vesselId                    portId  \
0 -34.74370  -57.85130  61e9f3a8b937134a3c4bfdf7  61d371c43aeaecc07011a37f   
1   8.89440  -79.47939  61e9f3d4b937134a3c4bff1f  634c4de270937fc01c3a7689   
2  39.19065  -76.47567  61e9f436b937134a3c4c0131  61d3847bb7b7526e1adf3d19   
3 -34.41189  151.02067  61e9f3b4b937134a3c4bfe77  61d36f770a1807568ff9a126   
4  35.88379   -5.91636  61e9f41bb937134a3c4c0087  634c4de270937fc01c3a74f3   

   port_latitude  port_longitude  
0       -33.5875      -71.618889  
1         8.9670

In [17]:
train_latitude_radians = np.deg2rad(train_data_preprocessed["latitude"])
train_longitude_radians = np.deg2rad(train_data_preprocessed["longitude"])
train_cog_radians = np.deg2rad(train_data_preprocessed["cog"])
train_heading_radians = np.deg2rad(train_data_preprocessed["heading"])

port_latitude_radians = np.deg2rad(train_data_preprocessed["port_latitude"])
port_longitude_radians = np.deg2rad(train_data_preprocessed["port_longitude"])

train_hour = np.deg2rad(train_data_preprocessed["time"].dt.hour * 360 / 24)
train_day = np.deg2rad(train_data_preprocessed["time"].dt.day * 360 / 30)
train_month = np.deg2rad(train_data_preprocessed["time"].dt.month * 360 / 12)


train_latitude_sin = np.sin(train_latitude_radians)
train_latitude_cos = np.cos(train_latitude_radians)
train_longitude_sin = np.sin(train_longitude_radians)
train_longitude_cos = np.cos(train_longitude_radians)

port_latitude_sin = np.sin(port_latitude_radians)
port_latitude_cos = np.cos(port_latitude_radians)
port_longitude_sin = np.sin(port_longitude_radians)
port_longitude_cos = np.cos(port_longitude_radians)

train_cog_sin = np.sin(train_cog_radians)
train_cog_cos = np.cos(train_cog_radians)

train_heading_sin = np.sin(train_heading_radians)
train_heading_cos = np.cos(train_heading_radians)

train_hour_sin = np.sin(train_hour)
train_hour_cos = np.cos(train_hour)

train_day_sin = np.sin(train_day)
train_day_cos = np.cos(train_day)

train_month_sin = np.sin(train_month)
train_month_cos = np.cos(train_month)


train_data_preprocessed["latitude_sin"] = train_latitude_sin
train_data_preprocessed["latitude_cos"] = train_latitude_cos
train_data_preprocessed["longitude_sin"] = train_longitude_sin
train_data_preprocessed["longitude_cos"] = train_longitude_cos
train_data_preprocessed["port_latitude_sin"] = train_latitude_sin
train_data_preprocessed["port_latitude_cos"] = train_latitude_cos
train_data_preprocessed["port_longitude_sin"] = train_longitude_sin
train_data_preprocessed["port_longitude_cos"] = train_longitude_cos
train_data_preprocessed["cog_sin"] = train_cog_sin
train_data_preprocessed["cog_cos"] = train_cog_cos
train_data_preprocessed["heading_sin"] = train_heading_sin
train_data_preprocessed["heading_cos"] = train_heading_cos

train_data_preprocessed["hour_sin"] = train_hour_sin
train_data_preprocessed["hour_cos"] = train_hour_cos
train_data_preprocessed["day_sin"] = train_day_sin
train_data_preprocessed["day_cos"] = train_day_cos
train_data_preprocessed["month_sin"] = train_month_sin
train_data_preprocessed["month_cos"] = train_month_cos


train_data_preprocessed["cog_sog_sin"] = train_data_preprocessed["cog_sin"]*train_data_preprocessed["sog"]
train_data_preprocessed["cog_sog_cos"] = train_data_preprocessed["cog_cos"]*train_data_preprocessed["sog"]

train_data_preprocessed = train_data_preprocessed.drop(
    columns=["latitude", "longitude", "cog", "heading", "portId","cog_sin","cog_cos","sog","port_latitude","port_longitude","hour_sin","hour_cos","day_sin","day_cos","month_sin","month_cos","rot","heading_sin","heading_cos","navstat"], axis=1
)
print(train_data_preprocessed.columns)

Index(['time', 'vesselId', 'seconds_to_eta', 'latitude_sin', 'latitude_cos',
       'longitude_sin', 'longitude_cos', 'port_latitude_sin',
       'port_latitude_cos', 'port_longitude_sin', 'port_longitude_cos',
       'cog_sog_sin', 'cog_sog_cos'],
      dtype='object')


In [18]:
def optimize_dataframe(df):
        """
        Downcasts numerical columns to reduce memory usage.
        """
        for col in df.select_dtypes(include=['float64']).columns:
            df[col] = pd.to_numeric(df[col], downcast='float')
        for col in df.select_dtypes(include=['int64']).columns:
            df[col] = pd.to_numeric(df[col], downcast='integer')
        return df

train_data_preprocessed = optimize_dataframe(train_data_preprocessed)

In [ ]:
def Last_known_location_training_data(
    data: pd.DataFrame, max_shift_lengths, max_instances_per_group=1000
) -> pd.DataFrame:
    """_summary_  Groups training data by vesselId, and propogates all data from last known location

    Args:
    data (_type_): _description_ the data to be altered

    Returns:
        _type_:? _description_ the altered data

    """
    all_test_data = pd.DataFrame()
    shift_length=1
    while shift_length<=max_shift_lengths:

        grouped_data = data.groupby("vesselId").apply(lambda x: x.sort_values("time"))

        grouped_data["time_diff"] = (
            grouped_data["time"].diff(-shift_length).dt.total_seconds().abs()
        )

        original_time_and_id = grouped_data[
            [
                "time",
                "vesselId",
                "latitude_sin",
                "latitude_cos",
                "longitude_sin",
                "longitude_cos",
            ]
        ]

        shifted_data = grouped_data.shift(shift_length)
        shifted_data[
            [
                "last_latitude_sin",
                "last_latitude_cos",
                "last_longitude_sin",
                "last_longitude_cos",
            ]
        ] = shifted_data[
            ["latitude_sin", "latitude_cos", "longitude_sin", "longitude_cos"]
        ]

        shifted_data[
            [
                "time",
                "vesselId",
                "latitude_sin",
                "latitude_cos",
                "longitude_sin",
                "longitude_cos",
            ]
        ] = original_time_and_id[
            [
                "time",
                "vesselId",
                "latitude_sin",
                "latitude_cos",
                "longitude_sin",
                "longitude_cos",
            ]
        ]
        

        # Drops all values with nan values
        result = shifted_data.dropna().reset_index(drop=True)

        # Step 6: Limit to max_instances_per_group per vesselId
        # Define a function to sample or take all if less than max_instances_per_group
        def sample_group(group):
            if len(group) > max_instances_per_group:
                return group.sample(n=max_instances_per_group, random_state=42)
            else:
                return group

        # Apply the sampling function to each group
        result = result.groupby('vesselId').apply(sample_group).reset_index(drop=True)

        all_test_data = pd.concat([all_test_data, result], ignore_index=True)

        prev_shift_length = shift_length
        shift_length = int(shift_length**(1.1))
        if shift_length == prev_shift_length:
            shift_length += 1
        print(shift_length)

    # Uncomment the line below if you want to remove the "time" column after processing
    # data = data.drop("time", axis=1)

    return all_test_data

In [20]:
print(train_data_preprocessed.columns)

train_data_shifted_df = Last_known_location_training_data(
    train_data_preprocessed, 400
)

# train_data_shifted_df = train_data_shifted_df.drop(columns=["time"], axis=1)

Index(['time', 'vesselId', 'seconds_to_eta', 'latitude_sin', 'latitude_cos',
       'longitude_sin', 'longitude_cos', 'port_latitude_sin',
       'port_latitude_cos', 'port_longitude_sin', 'port_longitude_cos',
       'cog_sog_sin', 'cog_sog_cos'],
      dtype='object')
2
3
4
5
6
7
8
9
10
11
12
13
14
15
17
19
22
25
29
34
40
48
58
71
87
108
136
173
223
292
387
521


In [21]:
def append_last_known_data_test(
    test_data: pd.DataFrame, known_data: pd.DataFrame
) -> pd.DataFrame:
    """_summary_  Groups training data by vesselId, and propogates all data from last known location

    Args:
    data (_type_): _description_ the data to be altered

    Returns:
        _type_:? _description_ the altered data
    """

    if not test_data["vesselId"].isin(known_data["vesselId"]).all():
        missing_vessels = test_data[
            ~test_data["vesselId"].isin(known_data["vesselId"])
        ]["vesselId"].unique()
        raise ValueError(
            f"The following vesselIds are missing in known_data: {missing_vessels}"
        )
    print(
        test_data[~test_data["vesselId"].isin(known_data["vesselId"])][
            "vesselId"
        ].unique()
    )

    grouped_data = (
        known_data.sort_values("time")
        .groupby("vesselId")
        .tail(1)
        .reset_index(drop=True)
    )
    original_time = test_data[["time"]]
    test_data = test_data.drop("time", axis=1)

    result = pd.merge(test_data, grouped_data, how="left", on="vesselId")

    result["time_diff"] = (original_time["time"] - result["time"]).dt.total_seconds()

    return result

In [22]:
test_data_with_last_known_df = append_last_known_data_test(
    test_data, train_data_preprocessed
)
test_data_with_last_known_df[
    [
        "last_latitude_sin",
        "last_latitude_cos",
        "last_longitude_sin",
        "last_longitude_cos",
    ]
] = test_data_with_last_known_df[
    [
        "latitude_sin",
        "latitude_cos",
        "longitude_sin",
        "longitude_cos",
    ]
]
test_data_with_last_known_df = test_data_with_last_known_df.drop(
    columns=[
        "latitude_sin",
        "latitude_cos",
        "longitude_sin",
        "longitude_cos",
    ],
    axis=1,
)

print(test_data_with_last_known_df.head())

[]
   ID                  vesselId  scaling_factor                time  \
0   0  61e9f3aeb937134a3c4bfe3d             0.3 2024-05-07 23:48:16   
1   1  61e9f473b937134a3c4c02df             0.3 2024-05-07 23:57:16   
2   2  61e9f469b937134a3c4c029b             0.3 2024-05-07 23:59:08   
3   3  61e9f45bb937134a3c4c0221             0.3 2024-05-07 23:52:34   
4   4  61e9f38eb937134a3c4bfd8d             0.3 2024-05-07 23:51:29   

   seconds_to_eta  port_latitude_sin  port_latitude_cos  port_longitude_sin  \
0       -133396.0           0.517228           0.855848           -0.989010   
1       -521836.0           0.255732           0.966748            0.863429   
2         45952.0           0.619491           0.785004            0.187086   
3        -81454.0          -0.688834           0.724919            0.124723   
4        101311.0           0.749340           0.662186           -0.106612   

   port_longitude_cos  cog_sog_sin  cog_sog_cos  time_diff  last_latitude_sin  \
0            0

In [ ]:
memory_usage_bytes = train_data_shifted_df.memory_usage(deep=True).sum()
# Convert bytes to gigabytes
memory_usage_gb = memory_usage_bytes / (1024 ** 3)
# Print the memory usage in gigabytes
print(f"DataFrame size: {memory_usage_gb:.2f} GB")

h2o.init(max_mem_size="6g")

train_data_shifted = h2o.H2OFrame(train_data_shifted_df)
test_data_with_last_known = h2o.H2OFrame(test_data_with_last_known_df)

del train_data_shifted_df
del test_data_with_last_known_df

DataFrame size: 3.11 GB
Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "23.0.1" 2024-10-15; OpenJDK Runtime Environment Homebrew (build 23.0.1); OpenJDK 64-Bit Server VM Homebrew (build 23.0.1, mixed mode, sharing)
  Starting server from /Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/7m/94fn3sg96x5fhts9_8f4x63c0000gn/T/tmprzh6zu9j
  JVM stdout: /var/folders/7m/94fn3sg96x5fhts9_8f4x63c0000gn/T/tmprzh6zu9j/h2o_iverringheim_started_from_python.out
  JVM stderr: /var/folders/7m/94fn3sg96x5fhts9_8f4x63c0000gn/T/tmprzh6zu9j/h2o_iverringheim_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Europe/Oslo
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.5
H2O_cluster_version_age:,2 months and 11 days
H2O_cluster_name:,H2O_from_python_iverringheim_96fpt5
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,5.984 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


H2OServerError: HTTP 500 Server Error:
<html>
<head>
<meta http-equiv="Content-Type" content="text/html;charset=ISO-8859-1"/>
<title>Error 500 java.lang.OutOfMemoryError: Java heap space</title>
</head>
<body><h2>HTTP ERROR 500 java.lang.OutOfMemoryError: Java heap space</h2>
<table>
<tr><th>URI:</th><td>/3/PostFile</td></tr>
<tr><th>STATUS:</th><td>500</td></tr>
<tr><th>MESSAGE:</th><td>java.lang.OutOfMemoryError: Java heap space</td></tr>
<tr><th>SERVLET:</th><td>water.api.PostFileServlet-207ea13</td></tr>
<tr><th>CAUSED BY:</th><td>java.lang.OutOfMemoryError: Java heap space</td></tr>
</table>
<h3>Caused by:</h3><pre>java.lang.OutOfMemoryError: Java heap space
	at water.fvec.UploadFileVec.readPut_impl(UploadFileVec.java:86)
	at water.fvec.UploadFileVec.readPut(UploadFileVec.java:61)
	at water.fvec.UploadFileVec.readPut(UploadFileVec.java:57)
	at water.api.PostFileServlet.doPost(PostFileServlet.java:40)
	at javax.servlet.http.HttpServlet.service(HttpServlet.java:707)
	at javax.servle

: 

In [ ]:
train_data_shifted_without_validation, validation_data_shifted = (
    train_data_shifted.split_frame(ratios=[0.9], seed=42)   
)               

In [ ]:
features_lat = [
    # "vesselId",
    "cog_sog_sin",
    "cog_sog_cos",
    #"rot",
    # "heading_sin",
    # "heading_cos",
    # "navstat",
    #"shippingLineId",
    "time_diff",
    "seconds_to_eta",
    "last_latitude_sin",
    "last_latitude_cos",
    "last_longitude_sin",
    "last_longitude_cos",
    "port_latitude_sin",
    "port_latitude_cos",
    "port_longitude_sin",
    "port_longitude_cos",
    # "hour_sin",
    # "hour_cos",
    #"day_sin",
    #"day_cos",
    # "month_sin",
    # "month_cos",
]
features_long = [
    # "vesselId",
    "cog_sog_sin",
    "cog_sog_cos",
    #"rot",
    #"shippingLineId",
    # "heading_sin",
    # "heading_cos",
    # "navstat",
    "time_diff",
    "seconds_to_eta",
    "last_latitude_sin",
    "last_latitude_cos",
    "last_longitude_sin",
    "last_longitude_cos",
    "port_latitude_sin",
    "port_latitude_cos",
    "port_longitude_sin",
    "port_longitude_cos",
    # "hour_sin",
    # "hour_cos",
    #"day_sin",
    #"day_cos",
    # "month_sin",
    # "month_cos",
    "latitude_sin",
    "latitude_cos",
]
target_long_sin = "longitude_sin"
target_long_cos = "longitude_cos"
target_lat_sin = "latitude_sin"
target_lat_cos = "latitude_cos"

In [ ]:
params_lat_sin = {
    'ntrees': 150,
    'max_depth': 20,
    'min_rows': 5,
    'mtries': -1,
    'sample_rate': 0.7,
    'col_sample_rate_per_tree': 0.8,
    'stopping_rounds': 5,
    'stopping_metric': "AUTO",
    'stopping_tolerance': 0.001,
    'seed': 42
}
params_lat_cos = {
    'ntrees': 150,
    'max_depth': 20,
    'min_rows': 5,
    'mtries': -1,
    'sample_rate': 0.7,
    'col_sample_rate_per_tree': 0.8,
    'stopping_rounds': 5,
    'stopping_metric': "AUTO",
    'stopping_tolerance': 0.001,
    'seed': 42
}
params_long_sin = {
    'ntrees': 150,
    'max_depth': 20,
    'min_rows': 5,
    'mtries': -1,
    'sample_rate': 0.7,
    'col_sample_rate_per_tree': 0.8,
    'stopping_rounds': 5,
    'stopping_metric': "AUTO",
    'stopping_tolerance': 0.001,
    'seed': 42
}
params_long_cos = {
    'ntrees': 150,
    'max_depth': 20,
    'min_rows': 5,
    'mtries': -1,
    'sample_rate': 0.7,
    'col_sample_rate_per_tree': 0.8,
    'stopping_rounds': 5,
    'stopping_metric': "AUTO",
    'stopping_tolerance': 0.001,
    'seed': 42
}

drf_lat_sin = h2o.estimators.H2ORandomForestEstimator(**params_lat_sin)
drf_lat_cos = h2o.estimators.H2ORandomForestEstimator(**params_lat_cos)
drf_long_sin = h2o.estimators.H2ORandomForestEstimator(**params_long_sin)
drf_long_cos = h2o.estimators.H2ORandomForestEstimator(**params_long_cos)


In [ ]:
training_params = {
    "drf_lat_sin": {
        "model": drf_lat_sin,
        "features": features_lat,
        "target": target_lat_sin,
    },
    "drf_lat_cos": {
        "model": drf_lat_cos,
        "features": features_lat,
        "target": target_lat_cos,
    },
    "drf_long_sin": {
        "model": drf_long_sin,
        "features": features_long,  # Ensure features_long is defined
        "target": target_long_sin,
    },
    "drf_long_cos": {
        "model": drf_long_cos,
        "features": features_long,  # Ensure features_long is defined
        "target": target_long_cos,
    },
}

In [ ]:
drf_lat_sin.train(
    x=features_lat,  # .append(latitude_sin)
    y=target_lat_sin,
    training_frame=train_data_shifted_without_validation,
    validation_frame=validation_data_shifted,
)

In [ ]:
drf_lat_cos.train(
    x=features_lat,  # .append(latitude_sin)
    y=target_lat_cos,
    training_frame=train_data_shifted_without_validation,
    validation_frame=validation_data_shifted,
)

In [ ]:
performance_lat_sin = drf_lat_sin.model_performance(test_data=validation_data_shifted, train_data=train_data_shifted_without_validation)
performance_lat_cos = drf_lat_cos.model_performance(test_data=validation_data_shifted,train_data=train_data_shifted_without_validation)


# Print the performance metrics
print(performance_lat_sin)
print(performance_lat_cos)

In [ ]:
drf_long_sin.train(
    x=features_long,
    y=target_long_sin,
    training_frame=train_data_shifted_without_validation,
    validation_frame=validation_data_shifted,
)
#MSE: 0.00023605270678827214

In [ ]:
drf_long_cos.train(
    x=features_long,  # .append("longitude_sin")
    y=target_long_cos,
    training_frame=train_data_shifted_without_validation,
    validation_frame=validation_data_shifted,
)

In [ ]:
performance_long_sin = drf_long_sin.model_performance(test_data=validation_data_shifted)
performance_long_cos = drf_long_cos.model_performance(test_data=validation_data_shifted)

print(performance_long_sin)
print(performance_long_cos)

In [ ]:
test_data_with_predicted_lat = test_data_with_last_known

lat_predictions_sin = drf_lat_sin.predict(test_data_with_last_known)
test_data_with_last_known["latitude_sin"] = lat_predictions_sin
lat_predictions_cos = drf_lat_cos.predict(test_data_with_last_known)
test_data_with_predicted_lat["latitude_cos"] = lat_predictions_cos

test_data_with_predicted_lat["latitude_sin"] = lat_predictions_sin
test_data_with_predicted_lat["latitude_cos"] = lat_predictions_cos

long_predictions_sin = drf_long_sin.predict(test_data_with_predicted_lat)
test_data_with_last_known["longitude_sin"] = long_predictions_sin
long_predictions_cos = drf_long_cos.predict(test_data_with_predicted_lat)
test_data_with_last_known["longitude_cos"] = long_predictions_cos

In [ ]:
# Convert sine and cosine values back to radians
lat_predictions_sin = lat_predictions_sin.as_data_frame()
lat_predictions_cos = lat_predictions_cos.as_data_frame()
long_predictions_sin = long_predictions_sin.as_data_frame()
long_predictions_cos = long_predictions_cos.as_data_frame()


lat_predictions_radians = np.arctan2(lat_predictions_sin, lat_predictions_cos)
long_predictions_radians = np.arctan2(long_predictions_sin, long_predictions_cos)

# Convert radians to degrees
lat_predictions_degrees = np.rad2deg(lat_predictions_radians)
long_predictions_degrees = np.rad2deg(long_predictions_radians)

# Print the first few rows to verify the conversion
print(lat_predictions_degrees.head())
print(long_predictions_degrees.head())

In [ ]:
def create_prediction_visualization_data(validation_data):
    lat_val_sin = drf_lat_sin.predict(validation_data)
    lat_val_cos = drf_lat_cos.predict(validation_data)
    long_val_sin = drf_long_sin.predict(validation_data)
    long_val_cos = drf_long_cos.predict(validation_data)

    lat_val_sin = lat_val_sin.as_data_frame()
    lat_val_cos = lat_val_cos.as_data_frame()
    long_val_sin = long_val_sin.as_data_frame()
    long_val_cos = long_val_cos.as_data_frame()

    validation_data = validation_data.as_data_frame()

    lat_val_radians = np.arctan2(lat_val_sin, lat_val_cos)
    long_val_radians = np.arctan2(long_val_sin, long_val_cos)

    evaluation_lat_radians = np.arctan2(
        validation_data["latitude_sin"], validation_data["latitude_cos"]
    )
    evaluation_long_radians = np.arctan2(
        validation_data["longitude_sin"], validation_data["longitude_cos"]
    )

    # Convert radians to degrees
    lat_val_degrees = np.rad2deg(lat_val_radians)
    long_val_degrees = np.rad2deg(long_val_radians)

    evaluation_lat_degrees = np.rad2deg(evaluation_lat_radians)
    evaluation_long_degrees = np.rad2deg(evaluation_long_radians)

    eval_predictions = pd.concat([lat_val_degrees, long_val_degrees], axis=1)

    eval_actual = pd.concat([evaluation_lat_degrees, evaluation_long_degrees], axis=1)

    eval_predictions.columns = ["latitude_predicted", "longitude_predicted"]
    eval_actual.columns = ["latitude", "longitude"]

    eval = pd.DataFrame()
    eval[["latitude_predicted", "longitude_predicted"]] = eval_predictions
    eval[["latitude", "longitude"]] = eval_actual
    eval[["vesselId", "time"]] = validation_data[["vesselId", "time"]]
    eval.to_csv("eval_predictions.csv")


# create_prediction_visualization_data(validation_data_shifted)

In [ ]:
predictions = pd.concat([lat_predictions_degrees, long_predictions_degrees], axis=1)
predictions.columns = ["latitude_predicted", "longitude_predicted"]

In [ ]:
predictions["ID"] = test_data["ID"]
predictions = predictions[["ID", "longitude_predicted", "latitude_predicted"]]

In [ ]:
print(predictions.columns)

In [ ]:
predictions.to_csv("drf_predictions.csv", index=False)